# Testing the `LT_signature` module

## Setup

In [1]:
import notebook_setup
import ipytest
import pytest
import warnings

# Suppress DeprecationWarnings from SageMath and other libraries
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', message=".*superseded by LazyCombinatorialSpecies.*")
warnings.filterwarnings('ignore', message=".*Importing .* from here is deprecated.*")

ipytest.autoconfig()

Notebook setup complete. Environment configured.


In [2]:
%preparse LT_signature
from gaknot.LT_signature import LT_signature_torus_knot, LT_signature_iterated_torus_knot, LT_signature_generalized_algebraic_knot

INFO: 

import_sage called with arguments:
	module_name: LT_signature
	package: gaknot
	path: /Users/wojtek/Library/CloudStorage/GoogleDrive-w.politarczyk@uw.edu.pl/My Drive/mat/git-projects/signature_function
INFO: 

import_sage called with arguments:
	module_name: signature
	package: gaknot
	path: /Users/wojtek/Library/CloudStorage/GoogleDrive-w.politarczyk@uw.edu.pl/My Drive/mat/git-projects/signature_function/gaknot
Successfully preparsed and reloaded: LT_signature


## Helper functions

In [3]:
def torus_knot_desc_to_txt(desc):
    if not isinstance(desc, (list, tuple)):
        raise TypeError(f'The desc parameter must be a list or a tuple. Got: {type(desc)}.')
    if len(desc) != 2:
        raise ValueError(f'The desc parameter must be of length 2. Got len(desc) = {len(desc)}.')
    return f'T({desc[0]}, {desc[1]})'

def it_torus_knot_desc_to_txt(desc):
    if not isinstance(desc, (list, tuple)):
        raise TypeError(f'The desc parameter must be a list or a tuple. Got: {type(desc)}.')
    return 'T(' + '; '.join([str(a) + ',' + str(b) for a,b in desc]) + ')'
    
def gaknot_desc_to_txt(desc):
    return ' # '.join(['-' * (int(1-sign) // 2) + it_torus_knot_desc_to_txt(knot_desc) for sign, knot_desc in desc])

## Tests

### Basic Torus Knots (Parametric)
Verifies total signature jump properties and the signature value at $\theta=1/2$ for a variety of torus knots $T(p,q)$ using parametric testing with 10 cases.

In [6]:
%%ipytest -vv -W ignore::DeprecationWarning

@pytest.mark.parametrize("p, q, expected_sig", [
    (2, 3, -2),
    (2, 5, -4),
    (2, 7, -6),
    (3, 4, -6),
    (3, 5, -8),
    (3, 7, -4),
    (4, 5, -8),
    (4, 7, -14),
    (5, 6, -12),
    (7, 8, -30)
])
def test_lt_signature_torus_knot_basic(p, q, expected_sig):
    sig = LT_signature_torus_knot(p, q)
    assert sig(0) == 0
    assert sig.total_sign_jump() == 0

    # Check the signature value at 1/2
    assert int(sig(0.5)) == expected_sig

======================================= test session starts ========================================
platform darwin -- Python 3.11.14, pytest-9.0.2, pluggy-1.6.0 -- /opt/homebrew/anaconda3/envs/sage_env/bin/python3
cachedir: .pytest_cache
rootdir: /Users/wojtek/Library/CloudStorage/GoogleDrive-w.politarczyk@uw.edu.pl/My Drive/mat/git-projects/signature_function/notebooks
plugins: anyio-4.12.1, langsmith-0.7.3
collecting ... collected 10 items

t_337677d018df4023be898051ad8f1759.py::test_lt_signature_torus_knot_basic[p0-q0-expected_sig0] PASSED [ 10%]
t_337677d018df4023be898051ad8f1759.py::test_lt_signature_torus_knot_basic[p1-q1-expected_sig1] PASSED [ 20%]
t_337677d018df4023be898051ad8f1759.py::test_lt_signature_torus_knot_basic[p2-q2-expected_sig2] PASSED [ 30%]
t_337677d018df4023be898051ad8f1759.py::test_lt_signature_torus_knot_basic[p3-q3-expected_sig3] PASSED [ 40%]
t_337677d018df4023be898051ad8f1759.py::test_lt_signature_torus_knot_basic[p4-q4-expected_sig4] PASSED [ 50%]
t_3376

### Error Handling
Ensures that the function correctly identifies invalid input parameters, such as non-coprime (p, q) pairs.

In [ ]:
def test_lt_signature_torus_knot_errors():
    with pytest.raises(ValueError, match="Parameteres p and q must be relatively prime"):
        LT_signature_torus_knot(3,6)

### Symmetry Property
Confirms that the signature function is symmetric with respect to its parameters: $\sigma_{T(p,q)} = \sigma_{T(q,p)}$.

In [ ]:
def test_lt_signature_torus_knot_symmetry():
    for i in range(3,11,2):
        sig1 = LT_signature_torus_knot(2,i)
        sig2 = LT_signature_torus_knot(i,2)
        assert sig1 == sig2

### Iterated Torus Knots
Tests the recursive computation of signatures for cabling structures, such as the (6,5)-cable of T(2,3).

In [ ]:
def test_lt_signature_iterated_torus_knot():
    # (6,5)-cable of T(2,3)
    iterated_sig = LT_signature_iterated_torus_knot([(2,3), (6,5)])
    assert iterated_sig.total_sign_jump() == 0

### Generalized Algebraic Knots
Verifies properties for connected sums and concordance inverses, including validation of known algebraically slice knots.

In [ ]:
def test_lt_signature_generalized_algebraic_knot():
    # T(2,3) # -T(2,3) should be zero
    desc_zero = [(1, [(2,3)]), (-1, [(2,3)])]
    sig_zero = LT_signature_generalized_algebraic_knot(desc_zero)
    assert sig_zero.is_zero_everywhere()
    
    # Complex combination known to be algebraically slice
    desc_complex = [
        (1, [(2,3), (5,2)]),
        (1, [(3,2)]),
        (1, [(5,3)]),
        (-1, [(6,5)])
    ]
    sig_complex = LT_signature_generalized_algebraic_knot(desc_complex)
    assert sig_complex.is_zero_everywhere()

## Run all tests

In [ ]:
ipytest.run()